# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime
import json
from shapely.geometry import shape
import geopandas as gpd
import geojson

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
468517,26215238,28,1,28,2,26042019,C9100,1,4,7,9,29,1.01201
788163,45357462,50,2,22,1,25012016,G379,1,43,10,1,1,1.00000
46885,1786567,3,1,3,2,2082016,A419,1,56,5,24,8,1.00000
655431,36937465,39,2,39,1,20052017,E6601,1,27,9,4,4,1.00000
723538,30618308,30,2,30,1,19052016,F312,1,44,11,27,4,1.05054


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title,resumen_cap
11,F11,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...,consumo psicotrópicos
64,F82,Trastornos de desarrollo específicos de funcio...,F80-89,Trastornos del desarrollo psicológico,desarrollo psicológico
51,F64,Trastornos de identidad de género,F60-69,Trastornos de la personalidad y del comportami...,T. de la personalidad/comportamiento


In [12]:
diagnos.code.unique()

array(['F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17',
       'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F40', 'F41',
       'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53',
       'F54', 'F55', 'F59', 'F60', 'F62', 'F63', 'F64', 'F66', 'F68',
       'F69', 'F65', 'F70', 'F71', 'F72', 'F73', 'F78', 'F79', 'F80',
       'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 'F90', 'F91', 'F92',
       'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [13]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [14]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [15]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [16]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
737240,28,1,28,2,31102017,F259,4,50,9,1.0,F25,F20-29
753286,28,1,28,2,28062017,F3173,1,57,16,1.0,F31,F30-39
730809,48,1,48,2,12012018,F250,1,20,32,1.0,F25,F20-29


In [17]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

In [18]:
dicc_res = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_res[r.cap] = r.resumen_cap

In [19]:
dicc_res

{'F00-F09': 'orgánicos/sintomáticos',
 'F10-F19': 'consumo psicotrópicos',
 'F20-29': 'esquizotípicos/delirantes',
 'F30-39': 'ánimo/afectivos',
 'F40-49': 'neuróticos/somatomorfos',
 'F50-59': 'factores físicos /fisiológicos',
 'F60-69': 'T. de la personalidad/comportamiento',
 'F70-79': 'deficiencia psíquica',
 'F80-89': 'desarrollo psicológico',
 'F90-F98': 'aparecen en niñez',
 'F99': 'sin espcificar'}

In [20]:
morbilidad['resumen'] = morbilidad.cap.map(dicc_res)

In [21]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,resumen
742217,28,1,28,2,10102017,F29,1,27,7,1.00000,F29,F20-29,esquizotípicos/delirantes
725421,31,1,31,2,30112017,F209,2,45,24,1.00000,F20,F20-29,esquizotípicos/delirantes
753678,28,1,21,1,26072019,F259,1,32,16,1.02096,F25,F20-29,esquizotípicos/delirantes


## Creamos fecha de ingreso

In [22]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [23]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [24]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [25]:
morbilidad['year']=morbilidad.fecha_ing.dt.year
morbilidad['month']=morbilidad.fecha_ing.dt.month
morbilidad['day']=morbilidad.fecha_ing.dt.day

In [26]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [27]:
# Cambiamos algunos datos categóricos que vienen como números 

In [28]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,resumen,fecha_al,fecha_ing,year,month,day
726929,28,1,28,2,F23,1,29,20,1.01195,F23,F20-29,esquizotípicos/delirantes,2018-05-16,2018-04-26,2018,4,26
743446,48,1,48,2,F29,2,71,16,1.00000,F29,F20-29,esquizotípicos/delirantes,2017-02-08,2017-01-23,2017,1,23
723496,48,1,48,2,F1420,1,34,11,1.00000,F14,F10-F19,consumo psicotrópicos,2019-07-15,2019-07-04,2019,7,4


In [29]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [30]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [31]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [32]:
#preparar provincias para geojsaon (nombre)

In [33]:
#diccionario
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [34]:
morbilidad['ProvHosp_nom'] = morbilidad.ProvHosp.map(cod_prov)
morbilidad['ProvResi_nom'] = morbilidad.ProvResi.map(cod_prov)

In [35]:
#prov resi 53 = extranjeros
morbilidad.ProvResi_nom.fillna('Extranjeros',inplace = True)

In [36]:
morbilidad.isna().sum()

ProvHosp        0
Sexo            0
ProvResi        0
DiagEntr        0
DiagPrin        0
MotivoAlta      0
EdadAnios       0
DiasEstancia    0
Factor          0
code            0
cap             0
resumen         0
fecha_al        0
fecha_ing       0
year            0
month           0
day             0
Entrada         0
Alta            0
SexNom          0
ProvHosp_nom    0
ProvResi_nom    0
dtype: int64

In [37]:
morbilidad.columns

Index(['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'DiagPrin', 'MotivoAlta',
       'EdadAnios', 'DiasEstancia', 'Factor', 'code', 'cap', 'resumen',
       'fecha_al', 'fecha_ing', 'year', 'month', 'day', 'Entrada', 'Alta',
       'SexNom', 'ProvHosp_nom', 'ProvResi_nom'],
      dtype='object')

In [38]:
morbilidad.rename({'EdadAnios':'Edad','DiasEstancia':'Dias'},axis=1,inplace = True)

In [39]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
749526,08,1,08,1,F250,1,59,30,1.04887,F25,...,2019-09-02,2019-08-03,2019,8,3,Ordinario,Curación,Hombre,Barcelona,Barcelona
755931,36,2,36,2,F600,1,54,47,1.00000,F60,...,2016-10-28,2016-09-11,2016,9,11,Urgente,Curación,Mujer,Pontevedra,Pontevedra
759485,04,1,04,2,F3340,1,61,10,1.00000,F33,...,2018-08-22,2018-08-12,2018,8,12,Urgente,Curación,Hombre,Almería,Almería


In [40]:
morbilidad.to_csv('../data/morbilidad_Fgen.csv',index=False)

In [41]:
## extras

In [42]:
dicc_poly = {pf['properties']['name']:pf['geometry'] for pf in provincias['features']}

In [43]:
morbilidad['geometry'] = morbilidad.ProvHosp_nom.map(dicc_poly)

In [44]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom,geometry
705534,48,1,48,2,F15229,1,35,15,1.0,F15,...,2018-01-02,2018,1,2,Urgente,Curación,Hombre,Vizcaya,Vizcaya,"{'type': 'MultiPolygon', 'coordinates': [[[[-2..."
775573,46,2,46,1,F5001,1,52,21,1.0,F50,...,2018-05-28,2018,5,28,Ordinario,Curación,Mujer,Valencia,Valencia,"{'type': 'MultiPolygon', 'coordinates': [[[[-0..."
779365,28,2,28,1,F603,1,47,15,1.0,F60,...,2018-06-17,2018,6,17,Ordinario,Curación,Mujer,Madrid,Madrid,"{'type': 'MultiPolygon', 'coordinates': [[[[-3..."


In [45]:
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"],
                                                    X["elev"])),
                            properties=dict(name=X["name"],
                                            description=X["description"])))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)